In [1]:
import os
from collections import Counter
import pandas as pd
import numpy as np
direc = r'contracts/'

In [2]:
files = os.listdir(direc)
contracts_ = [direc + mail for mail in files]
word = []
c = len(contracts_)
for contract in contracts_:
    f = open(contract , encoding="iso8859_1")
    blob = f.read()
    word += blob.split(' ')
for i in range(len(word)):
    if not word[i].isalpha():
        word[i] = ''
        
dictionary  = Counter(word)
del dictionary['']


In [3]:
def make_dataset(dictionary):
    direc = r'contracts/'
    files = os.listdir(direc)
    contracts_ = [direc + email for email in files]
    feature_set = []
    labels = []
    file_ = []
    c = len(contracts_)

    for contract in contracts_:
        data = []
        f = open(contract, encoding="iso8859_1")
        file_.append(contract)
        words = f.read().split(' ')
        for entry in dictionary:
            data.append(words.count(entry[0]))
        feature_set.append(data)

        if "AMENDMENT TO EMPLOYMENT AGREEMENT" in contract:
            labels.append(0)
        else:
            labels.append(1)
    return feature_set, labels , file_

In [4]:
d = dictionary
a,b,c = make_dataset(d)
features , labels = a,b
df_ = {'file':c,'label':labels}
df = pd.DataFrame(df_)
df.drop(df.tail(2).index,inplace=True)

In [5]:
df.tail()

,file,label
996,contracts/948846_2004-10-29_LEONARD EMPLOYMENT...,1
997,contracts/948850_2003-04-25_EMPLOYMENT AGREEME...,1
998,contracts/949874_2011-01-05_AMENDMENT TO EMPLO...,0
999,contracts/949874_2011-05-13_AMENDMENT TO DANIE...,1
1000,contracts/contracts - Shortcut.lnk,1


In [6]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(df['file'], 
                                                    df['label'], 
                                                    random_state=0)

In [7]:
def add_feature(X, feature_to_add):
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
import operator
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
vectorizer = TfidfVectorizer()
vectorizer = TfidfVectorizer(min_df=10, ngram_range=[1,3])
X_train_transformed = vectorizer.fit_transform(X_train)
X_train_transformed_with_length = add_feature(X_train_transformed, [X_train.str.len(),
                                                                        X_train.apply(lambda x: len(''.join([a for a in x if a.isdigit()])))])
X_test_transformed = vectorizer.transform(X_test)
X_test_transformed_with_length = add_feature(X_test_transformed, [X_test.str.len(),
                                                                      X_test.apply(lambda x: len(''.join([a for a in x if a.isdigit()])))])

clf = LogisticRegression(C=100)

clf.fit(X_train_transformed_with_length, y_train)

y_predicted = clf.predict(X_test_transformed_with_length)
roc_auc_score(y_test, y_predicted)

0.9812254650964329

In [9]:
#df = df['file'][df['label'] == 1]

X_test_transformed = vectorizer.transform(df['file'])
X_test_transformed_with_length1 = add_feature(X_test_transformed, [df['file'].str.len(),
                                                                      df['file'].apply(lambda x: len(''.join([a for a in x if a.isdigit()])))])

In [10]:
y_predict_ = clf.predict(X_test_transformed_with_length1)

In [11]:
y_predict_

array([0, 0, 0, ..., 0, 1, 1], dtype=int64)

In [12]:
df['y_predict'] = y_predict_
df.head()

,file,label,y_predict
0,contracts/1000736_2005-05-10_AMENDMENT TO EMPL...,0,0
1,contracts/1001250_2013-02-27_AMENDMENT TO EMPL...,0,0
2,contracts/1001250_2013-02-27_AMENDMENT TO EMPL...,0,0
3,contracts/1001250_2013-02-27_AMENDMENT TO EMPL...,0,0
4,contracts/1001425_1999-05-14_DOUGLAS REED EMPL...,1,1


In [13]:
df1 = df.loc[df['y_predict'] == 1]

In [14]:
df1 = df1.drop('label',axis = 1)

In [15]:
df1.reset_index(inplace = True)

In [30]:
df1.file[17]

'contracts/100493_2005-02-09_TYSON FOODS, INC. SECOND AMENDMENT TO BOND EMPLOYMENT AGREEMENT.txt'

In [60]:
import glob
import os

x = glob.glob('contracts/*.txt')


In [92]:
c1 = []
i = 0
for file in x:
    f = open(file,encoding="utf8")
    blob = f.read()
    c1.append(blob)


In [93]:
import spacy
nlp = spacy.load('en')

In [94]:
test1 = re.sub(r'\([^)]*\)', '', c[0])
test2 = ' '.join(test1.split())

In [95]:
import re
test3 = re.sub('[!@#/$]', '', test2)


In [96]:
text1 = nlp(test3)


In [97]:
import nltk
for sent in nltk.sent_tokenize(c[0]):
   for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
      if hasattr(chunk, 'label'):
         print(chunk.label(), ' '.join(c[0] for c in chunk))

GPE Amendment
ORGANIZATION Employment
ORGANIZATION AMENDMENT
PERSON Caremark Rx
ORGANIZATION Delaware Corporation
PERSON Richard Scardina
PERSON Burr Ridge
GPE Illinois
GPE WHEREAS
PERSON Employer
PERSON Officer
PERSON Employer
ORGANIZATION Senior
ORGANIZATION WHEREAS
PERSON Employer
ORGANIZATION Executive
ORGANIZATION WHEREAS
ORGANIZATION Parties
ORGANIZATION Employment
PERSON Executive
ORGANIZATION THEREFORE
ORGANIZATION Employment Agreement
PERSON Officer
PERSON Executive
ORGANIZATION Employment
GPE Employer
ORGANIZATION Employment Agreement
PERSON Officer
PERSON Executive
GPE Operations
GPE Chicago
ORGANIZATION Employer
ORGANIZATION Employer
GPE Subsection
ORGANIZATION Employment
ORGANIZATION Term
PERSON Officer
GPE Employer
PERSON Base Salary
ORGANIZATION Employer
PERSON Officer
PERSON Officer
ORGANIZATION Employer
GPE Employer
ORGANIZATION Employment
ORGANIZATION Amendment
PERSON Caremark Rx
PERSON Richard Scardina
PERSON Richard Scardina Chairman
